# Simulating orbits of two planets

**PHAS0007 Computing final assignment: supplementary notebook**

**Completed version, with code from part A solution.**

Author: Dr Ben Waugh

Last updated: 2022-12-16

## Introduction

In part A of the final assignment, you were asked to complete three functions:
- `force`, to calculate the gravitational force between two objects;
- `move_planet`, to update the position and velocity of a planet after a time step, using Euler's method;
- `trajectory`, to calculate the trajectory of a planet over a given time interval.

You were given functions:
- `animate_with_trail`, to update the position of a planet for each frame of an animation;
- `create_animation`, to create an animation of an object with a given trajectory.

Using these functions, you should have created an animation showing the orbit of a single planet around a star. In part B you need to create an animation showing the orbits of two planets, first treating them independently and then including the gravitational force between them. This notebook will guide you through some of the steps needed for this. You may copy or adapt this code in your own notebook, as long as you reference it appropriately.

## Code from part A

To use this notebook, you will need to set up Matplotlib, import the required packages, and copy the functions `force`, `move_planet` and `trajectory` from your solution to part A into the cell below.

In [ ]:
##################
# YOUR CODE HERE #
##################


## Two independent planets

To start with, we will not take into account the gravitational interaction between the two planets, so they move independently. This means we can use the existing function `trajectory` for each planet separately.

To create an animation of the two planets, we need new versions of some of the functions we used in part A. The animation function can be amended to take a list of trajectories, and corresponding lists of planet and trail objects, instead of just one of each.

In [ ]:
def animate_multiple(i, planets, trails, trajectories):
    """
    Update display for motion of multiple bodies with trails.
    
    Arguments:
        i:       frame number (from 0 at time = 0)
        planets: list of Line2D objects containing coordinates of each body to move
        trails:  list of Line2D objects containing coordinates of trails
        trajectories: list of trajectories, each of the form [x_arr, y_arr]
                 where x_arr and y_arr are arrays of x and y coordinates at
                 each time step
    
    Note that all trajectories must have the same number of points.
    """
    for j in range(len(trajectories)):    # for each trajectory ...
        trajectory = trajectories[j]
        x_arr, y_arr = trajectory         # get arrays of x and y coordinats
        x, y = x_arr[i], y_arr[i]
        planets[j].set_data([x],[y])
        trails[j].set_data(x_arr[:i+1],y_arr[:i+1]) # Trail has all points up to this one

A new function can be used to obtain the required Line2D objects for planets and their trails, and ensure that they have matching colours.

In [ ]:
def get_bodies_and_trails(axes, n):
    """
    Get the required number of Line2D objects to represent the bodies
    being animated, and the trails left behind them.
    
    Arguments:
        axes: the axes to be used for plotting the bodies
        n:    number of bodies in simulation
    
    Returns: (bodies, trails) where
        bodies is a list of Line2D objects to be plotted as discs
        trails is a list of Line2D objects to be plotted as lines
    """
    bodies = []
    trails = []
    for i in range(n):              # repeat n times
        body,  = axes.plot([],[],'o')           # Line2D with disc markers
        col    = body.get_color()               # get colour of body ...
        trail, = axes.plot([],[],'-',color=col) # and set trail (line) to same colour
        bodies.append(body)         # add Line2D objects to lists
        trails.append(trail)
    return bodies, trails

Then the function that creates the animation also needs to be able to handle more than a single planet.

In [ ]:
def create_animation_planets(trajectories, dt, fps, speedup):
    """
    Create an animation of objects with given trajectories.
    
    Arguments:
        trajectories: list of trajectories, each of the form [x_arr, y_arr]
                 where x_arr and y_arr are arrays of x and y coordinates at
                 each time step
        dt:           time step in simulation [s]
        fps:          number of frames per second for animation
        speedup:      ratio of simulated time to screen time
    """
    # Find range of coordinates to include full trajectories 
    r_max = 1.05 * np.max(np.abs(trajectories))

    # Create and configure figure and axes
    plt.ioff()
    fig, axes = plt.subplots()
    axes.set_xlim(-r_max,r_max)
    axes.set_ylim(-r_max,r_max)
    axes.set_aspect('equal')

    # Create Line2D objects to represent bodies and trails
    bodies, trails = get_bodies_and_trails(axes, 2)
    body = bodies[0]
    trail = trails[0]

    # Get arrays of coordinates to display (subset of provided data)
    frame_interval = 1/fps                  # time between frames [s]
    skip = round(frame_interval*speedup/dt) # number of calculated points per frame
    plot_trajectories = []
    for trajectory in trajectories:
        x_arr, y_arr = trajectory
        x_plot = x_arr[::skip]  # take every Nth value (N = skip)
        y_plot = y_arr[::skip]
        plot_trajectory = [x_plot, y_plot]
        plot_trajectories.append(plot_trajectory)
        n_frames = len(x_plot)
    
    # Create and return animation object
    ani = animation.FuncAnimation(fig,animate_multiple, frames=n_frames, interval=frame_interval*1000,
                                  fargs=[bodies, trails, plot_trajectories])
    return ani

Using these functions, we can create an animation with two bodies orbiting independently.

In [ ]:
# Constants and parameters
G       = 6.6743e-11 # gravitational constant [m^3 kg^-1 s^-2]
YEAR    = 3.154e7    # one year in seconds
M_STAR  = 2.5e30     # mass of star [kg]
dt = YEAR/10000      # time step for Euler's method

# Initial position (x,y) [m] and velocity (vx,vy) [m/s] of each planet
pos_a = np.array([3e11, 0])
vel_a = np.array([0,2e4])
pos_b = np.array([0,2e11])
vel_b = np.array([-3e4,0])

# Calculate trajectories (lists of x and y coordinates) [m]
x_a, y_a = trajectory(pos_a,vel_a,M_STAR,dt,2*YEAR)
x_b, y_b = trajectory(pos_b,vel_b,M_STAR,dt,2*YEAR)

# Put the results into a list of trajectories, with each trajectory being
# a list containing the lists of x and y coordinates.
traj_a = [x_a, y_a]
traj_b = [x_b, y_b]
trajectories = [traj_a, traj_b]

# One year is represented by 2 seconds in the animation
speedup = YEAR/2

# Create and display the animation
ani = create_animation_planets(trajectories, dt, 50, speedup)
HTML(ani.to_jshtml())

## Interacting planets

To allow for the gravitational interactions between planets, we can no longer calculate the trajectories separately. Instead, at each time step we must update the position and velocity of each planet taking into account the position of the other planet as well as the star.

You will need to create functions similar to `move_planet` and `trajectory` in part A. This time the arguments will need to include the initial positions and velocities of both planets, as well as their masses.